In [1]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import numpy as np
from typing import Dict, List, Optional
from datetime import datetime
import pandas as pd
from collections import defaultdict

class AnomalyDetector:
    def __init__(self, data_loader, transaction_analyzer):
        self.data_loader = data_loader
        self.transaction_analyzer = transaction_analyzer
        self.isolation_forest = None
        self.scaler = StandardScaler()
        self.global_features = None
        self.contract_models = None
        self.trained = False

    def prepare_global_features(self):
        """Підготовка характеристик для звичайних гаманців"""
        features = []
        addresses = []
        feature_columns = [
            'tx_count', 
            'avg_value', 
            'std_value', 
            'unique_contacts',
            'avg_time_diff', 
            'std_time_diff', 
            'contract_interaction_ratio'
        ]
    
        # Беремо тільки не-контрактні адреси
        for address in self.data_loader.get_non_contract_addresses():
            feature_vector = self._extract_wallet_features(address)
            if feature_vector:
                features.append(feature_vector)
                addresses.append(address)
            
        if features:
            self.global_features = pd.DataFrame(
                features,
                columns=feature_columns,  # Додаємо назви колонок
                index=addresses
            )
            
    def train_model(self):
        """Тренування моделей для виявлення аномалій"""
        if self.global_features is None:
            self.prepare_global_features()
            
        # Модель для звичайних гаманців
        scaled_features = self.scaler.fit_transform(self.global_features)
        self.isolation_forest = IsolationForest(
            contamination=0.1,
            random_state=42
        )
        self.isolation_forest.fit(scaled_features)
        
        # Підготовка моделей для різних типів контрактів
        self._prepare_contract_models()
        
        self.trained = True
        
    def detect_anomalies(self, address: str) -> Dict:
        """Виявлення аномалій для адреси"""
        if not self.trained:
            self.train_model()
            
        # Визначаємо тип адреси
        if self.data_loader.is_contract(address):
            return self._detect_contract_anomalies(address)
        else:
            return self._detect_wallet_anomalies(address)
            
    def _prepare_contract_models(self):
        """Підготовка моделей для аналізу контрактів"""
        contract_features = defaultdict(list)
        contract_addresses = defaultdict(list)
        
        # Збір характеристик контрактів
        for address in self.data_loader.get_contract_addresses():
            features = self._extract_contract_features(address)
            if features:
                label = self.data_loader.get_address_info(address).get('label', 'unknown')
                contract_features[label].append(features)
                contract_addresses[label].append(address)
                
        # Створення окремих моделей для кожного типу контракту
        self.contract_models = {}
        for label in contract_features:
            if len(contract_features[label]) > 10:  # Мінімальна кількість для навчання
                features_array = np.array(contract_features[label])
                scaler = StandardScaler()
                scaled_features = scaler.fit_transform(features_array)
                
                model = IsolationForest(contamination=0.1, random_state=42)
                model.fit(scaled_features)
                
                self.contract_models[label] = {
                    'model': model,
                    'scaler': scaler
                }
                
    def _extract_wallet_features(self, address: str) -> Optional[List[float]]:
        """Витяг характеристик для звичайного гаманця"""
        transactions = self.data_loader.get_node_transactions(address)
        if not transactions:
            return None
            
        # Базові метрики
        tx_values = [float(tx['value$']) for tx in transactions]
        timestamps = [datetime.strptime(tx['timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ') 
                     for tx in transactions]
        
        # Часові різниці
        time_diffs = [(timestamps[i] - timestamps[i-1]).total_seconds() / 3600 
                     for i in range(1, len(timestamps))]
        
        # Взаємодії з контрактами
        contract_interactions = sum(1 for tx in transactions 
                                 if self.data_loader.is_contract(tx['to']))
        
        # Вектор характеристик
        return [
            len(transactions),                          # Кількість транзакцій
            np.mean(tx_values),                        # Середнє значення
            np.std(tx_values),                         # Стандартне відхилення
            len(set(tx['to'] for tx in transactions)), # Унікальні контакти
            np.mean(time_diffs) if time_diffs else 0,  # Середній час між транзакціями
            np.std(time_diffs) if time_diffs else 0,   # Стандартне відхилення часу
            contract_interactions / len(transactions)   # Частка взаємодій з контрактами
        ]
        
    def _extract_contract_features(self, address: str) -> Optional[List[float]]:
        """Витяг характеристик для контракту"""
        transactions = self.data_loader.get_node_transactions(address)
        if not transactions:
            return None
            
        # Вхідні та вихідні транзакції
        incoming = [tx for tx in transactions if tx['to'] == address]
        outgoing = [tx for tx in transactions if tx['from'] == address]
        
        # Значення транзакцій
        in_values = [float(tx['value$']) for tx in incoming]
        out_values = [float(tx['value$']) for tx in outgoing]
        
        return [
            len(incoming),                            # Кількість вхідних
            len(outgoing),                           # Кількість вихідних
            np.mean(in_values) if in_values else 0,  # Середнє вхідне значення
            np.mean(out_values) if out_values else 0,# Середнє вихідне значення
            np.std(in_values) if in_values else 0,   # Стандартне відхилення вхідних
            np.std(out_values) if out_values else 0, # Стандартне відхилення вихідних
            len(set(tx['from'] for tx in incoming)), # Унікальні відправники
            len(set(tx['to'] for tx in outgoing))    # Унікальні отримувачі
        ]
        
    def _detect_wallet_anomalies(self, address: str) -> Dict:
        """Виявлення аномалій для звичайного гаманця"""
        features = self._extract_wallet_features(address)
        if not features:
            return {'anomalies': [], 'anomaly_score': 0}
            
        # Передбачення аномальності
        scaled_features = self.scaler.transform([features])
        is_anomaly = self.isolation_forest.predict(scaled_features)[0] == -1
        anomaly_score = -self.isolation_forest.score_samples(scaled_features)[0]
        
        # Збір аномалій
        anomalies = []
        if is_anomaly:
            anomalies.extend(self._detect_value_anomalies(address))
            anomalies.extend(self._detect_temporal_anomalies(address))
            anomalies.extend(self._detect_pattern_anomalies(address))
            
        return {
            'anomalies': anomalies,
            'anomaly_score': float(anomaly_score),
            'is_anomaly': is_anomaly
        }
        
    def _detect_contract_anomalies(self, address: str) -> Dict:
        """Виявлення аномалій для контракту"""
        features = self._extract_contract_features(address)
        if not features:
            return {'anomalies': [], 'anomaly_score': 0}
            
        label = self.data_loader.get_address_info(address).get('label', 'unknown')
        
        # Якщо є модель для цього типу контракту
        if label in self.contract_models:
            model_data = self.contract_models[label]
            scaled_features = model_data['scaler'].transform([features])
            is_anomaly = model_data['model'].predict(scaled_features)[0] == -1
            anomaly_score = -model_data['model'].score_samples(scaled_features)[0]
        else:
            is_anomaly = False
            anomaly_score = 0
            
        # Специфічні аномалії для контрактів
        anomalies = []
        if is_anomaly:
            anomalies.extend(self._detect_contract_specific_anomalies(address))
            
        return {
            'anomalies': anomalies,
            'anomaly_score': float(anomaly_score),
            'is_anomaly': is_anomaly,
            'contract_type': label
        }
        
    def _detect_contract_specific_anomalies(self, address: str) -> List[Dict]:
        """Виявлення специфічних аномалій для контрактів"""
        anomalies = []
        transactions = self.data_loader.get_node_transactions(address)
        
        # Аналіз вхідних/вихідних патернів
        incoming = defaultdict(list)
        outgoing = defaultdict(list)
        
        for tx in transactions:
            if tx['to'] == address:
                incoming[tx['from']].append(float(tx['value$']))
            else:
                outgoing[tx['to']].append(float(tx['value$']))
                
        # Перевірка на незвичні патерни взаємодії
        for sender, values in incoming.items():
            if len(values) > 100 and np.std(values) / np.mean(values) > 2:
                anomalies.append({
                    'type': 'contract_interaction',
                    'subtype': 'unusual_input_pattern',
                    'details': {
                        'sender': sender,
                        'interaction_count': len(values),
                        'value_volatility': np.std(values) / np.mean(values)
                    }
                })
                
        return anomalies
        
    def _detect_value_anomalies(self, address: str) -> List[Dict]:
        """Виявлення аномалій в значеннях транзакцій"""
        transactions = self.data_loader.get_node_transactions(address)
        anomalies = []
        
        if not transactions:
            return anomalies
            
        values = [float(tx['value$']) for tx in transactions]
        mean_value = np.mean(values)
        std_value = np.std(values)
        
        for tx in transactions:
            value = float(tx['value$'])
            z_score = (value - mean_value) / std_value if std_value > 0 else 0
            
            if abs(z_score) > 3:  # 3 сигми
                anomalies.append({
                    'type': 'value',
                    'subtype': 'extreme_value',
                    'timestamp': tx['timestamp'],
                    'details': {
                        'value': value,
                        'z_score': z_score,
                        'to': tx['to'],
                        'is_contract': self.data_loader.is_contract(tx['to'])
                    }
                })
                
        return anomalies
        
    def _detect_temporal_anomalies(self, address: str) -> List[Dict]:
        """Виявлення часових аномалій"""
        transactions = self.data_loader.get_node_transactions(address)
        anomalies = []
        
        if len(transactions) < 2:
            return anomalies
            
        timestamps = [datetime.strptime(tx['timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ') 
                     for tx in transactions]
        time_diffs = [(timestamps[i] - timestamps[i-1]).total_seconds() / 3600 
                     for i in range(1, len(timestamps))]
        
        mean_diff = np.mean(time_diffs)
        std_diff = np.std(time_diffs)
        
        for i, diff in enumerate(time_diffs, 1):
            z_score = (diff - mean_diff) / std_diff if std_diff > 0 else 0
            
            if abs(z_score) > 3:
                anomalies.append({
                    'type': 'temporal',
                    'subtype': 'unusual_timing',
                    'timestamp': transactions[i]['timestamp'],
                    'details': {
                        'time_diff': diff,
                        'z_score': z_score,
                        'usual_time_diff': mean_diff
                    }
                })
                
        return anomalies
        
    def _detect_pattern_anomalies(self, address: str) -> List[Dict]:
        """Виявлення аномалій в патернах поведінки"""
        patterns = self.transaction_analyzer.analyze_wallet_patterns(address)
        anomalies = []
        
        if not patterns:
            return anomalies
            
        # Аномалії в послідовностях
        if 'sequence' in patterns:
            seq_patterns = patterns['sequence']
            if seq_patterns['avg_sequence_length'] > 5:  # Незвично довгі послідовності
                anomalies.append({
                    'type': 'pattern',
                    'subtype': 'long_sequence',
                    'details': {
                        'avg_length': seq_patterns['avg_sequence_length']
                    }
                })
                
        # Аномалії в контрактних взаємодіях
        if 'contract_usage' in patterns:
            contract_usage = patterns['contract_usage']
            if contract_usage['unique_contracts'] > 20:  # Багато різних контрактів
                anomalies.append({
                    'type': 'pattern',
                    'subtype': 'high_contract_diversity',
                    'details': {
                        'unique_contracts': contract_usage['unique_contracts']
                    }
                })
                
        return anomalies

In [2]:
%run data_loader.ipynb
%run transaction_pattern.ipynb

import json
from datetime import datetime
from pprint import pprint

def analyze_anomalies(detector, address: str):
    """Детальний аналіз аномалій для адреси"""
    print(f"\nAnalyzing anomalies for address: {address}")
    print("=" * 50)
    
    # Отримуємо результати аналізу
    results = detector.detect_anomalies(address)
    
    # Базова інформація
    print(f"\nAnomaly Score: {results['anomaly_score']:.4f}")
    print(f"Is Anomalous: {results['is_anomaly']}")
    
    if 'contract_type' in results:
        print(f"Contract Type: {results['contract_type']}")
        
    # Деталізація аномалій
    if results['anomalies']:
        print("\nDetected Anomalies:")
        anomalies_by_type = {}
        
        # Групуємо аномалії за типами
        for anomaly in results['anomalies']:
            anomaly_type = anomaly['type']
            if anomaly_type not in anomalies_by_type:
                anomalies_by_type[anomaly_type] = []
            anomalies_by_type[anomaly_type].append(anomaly)
            
        # Виводимо аномалії по групах
        for anomaly_type, anomalies in anomalies_by_type.items():
            print(f"\n{anomaly_type.upper()} ANOMALIES:")
            for anomaly in anomalies:
                print(f"\nSubtype: {anomaly['subtype']}")
                if 'timestamp' in anomaly:
                    print(f"Time: {anomaly['timestamp']}")
                print("Details:")
                pprint(anomaly['details'])
    else:
        print("\nNo anomalies detected")
        
    return results

def test():
    # Ініціалізація
    print("Initializing components...")
    loader = DataLoader()
    loader.load_addresses("data/addresses.json")
    loader.load_tokens("data/tokens.json")
    loader.load_nodes("data/nodes_new.csv")
    
    tx_analyzer = TransactionPatternAnalyzer(loader)
    tx_analyzer.build_interaction_graph()
    
    detector = AnomalyDetector(loader, tx_analyzer)
    
    # Підготовка та тренування моделі
    print("Preparing and training models...")
    detector.prepare_global_features()
    detector.train_model()
    
    # Аналіз різних типів адрес
    contract_address = "0x9696f59e4d72e237be84ffd425dcad154bf96976"

    addresses = [
        '0xeba88149813bec1cccccfdb0dacefaaa5de94cb1', # binance
        '0x39cf2e49ea4d620e77d67088a8d815348e0abdf6', # normal
        '0xa1b1bbb8070df2450810b8eb2425d543cfcef79b', # fund
    ]

    def count_anomalies_by_type(results):
        counts = defaultdict(int)
        for anomaly in results['anomalies']:
            counts[anomaly['type']] += 1
        return dict(counts)
    
    # Аналіз контракту
    print("\nAnalyzing contract...")
    contract_results = analyze_anomalies(detector, contract_address)
    contract_anomaly_counts = count_anomalies_by_type(contract_results)
    print("\nContract anomalies by type:")
    pprint(contract_anomaly_counts)

    

    for address in addresses:
        # Аналіз гаманця
        print("\nAnalyzing wallet...")
        wallet_results = analyze_anomalies(detector, address)
    
        # Порівняльний аналіз
        print("\nComparative Analysis:")
        print("-" * 30)
        print(f"Contract Anomaly Score: {contract_results['anomaly_score']:.4f}")
        print(f"Wallet Anomaly Score: {wallet_results['anomaly_score']:.4f}")

        wallet_anomaly_counts = count_anomalies_by_type(wallet_results)
    
        print("\nWallet anomalies by type:")
        pprint(wallet_anomaly_counts)
    
        # Додатковий аналіз для гаманця
        if not loader.is_contract(address):
            # Часові патерни аномалій
            temporal_anomalies = [
                a for a in wallet_results['anomalies']
                if a['type'] == 'temporal'
            ]
            if temporal_anomalies:
                print("\nTemporal Anomaly Analysis:")
                for anomaly in temporal_anomalies:
                    print(f"\nTimestamp: {anomaly['timestamp']}")
                    print(f"Time difference: {anomaly['details']['time_diff']:.2f} hours")
                    print(f"Z-score: {anomaly['details']['z_score']:.2f}")
        
            # Аномалії значень
            value_anomalies = [
                a for a in wallet_results['anomalies']
                if a['type'] == 'value'
            ]
            if value_anomalies:
                print("\nValue Anomaly Analysis:")
                for anomaly in value_anomalies:
                    print(f"\nValue: {anomaly['details']['value']}")
                    print(f"Z-score: {anomaly['details']['z_score']:.2f}")
                    print(f"To contract: {anomaly['details']['is_contract']}")

In [3]:
test()

Initializing components...
Loaded 1733 addresses
Loaded 5825 tokens
Loaded 1025 nodes
Contracts: 51
Non-contracts: 974
Preparing and training models...

Analyzing contract...

Analyzing anomalies for address: 0x9696f59e4d72e237be84ffd425dcad154bf96976

Anomaly Score: 0.5541
Is Anomalous: True

Detected Anomalies:

VALUE ANOMALIES:

Subtype: extreme_value
Time: 2021-05-27T23:09:02.025Z
Details:
{'is_contract': False,
 'to': '0x96181a7e0195cef37f71c6442d8a2df146d2a347',
 'value': 854000.0,
 'z_score': 4.187555379264425}

Subtype: extreme_value
Time: 2021-05-29T21:02:02.991Z
Details:
{'is_contract': False,
 'to': '0x3ba76787d7d9bd0d40370905f746e75d1f0e578e',
 'value': 654185.0,
 'z_score': 3.1083255811949293}

Subtype: extreme_value
Time: 2021-05-30T01:45:57.486Z
Details:
{'is_contract': False,
 'to': '0x5d2d9201f8c2a42c5ddb06e70de1d49e01e8281b',
 'value': 637000.0,
 'z_score': 3.0155069035189577}

Subtype: extreme_value
Time: 2021-06-23T16:40:55.236Z
Details:
{'is_contract': False,
 'to'

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Anomaly Score: 0.5283
Is Anomalous: True

Detected Anomalies:

VALUE ANOMALIES:

Subtype: extreme_value
Time: 2024-08-29T15:16:52.541Z
Details:
{'is_contract': False,
 'to': '0xeba88149813bec1cccccfdb0dacefaaa5de94cb1',
 'value': 19318.614,
 'z_score': 4.315940532309118}

Subtype: extreme_value
Time: 2024-09-07T07:27:42.491Z
Details:
{'is_contract': False,
 'to': '0xf1999b090747cc2d4c9ea287ada9321d092044f0',
 'value': 42890.61742,
 'z_score': 9.894156983729106}

Subtype: extreme_value
Time: 2024-09-07T08:00:22.155Z
Details:
{'is_contract': False,
 'to': '0xeba88149813bec1cccccfdb0dacefaaa5de94cb1',
 'value': 20998.845,
 'z_score': 4.713560201684928}

Subtype: extreme_value
Time: 2024-09-08T12:28:39.632Z
Details:
{'is_contract': False,
 'to': '0xeba88149813bec1cccccfdb0dacefaaa5de94cb1',
 'value': 27999.384,
 'z_score': 6.370208604572662}

Subtype: extreme_value
Time: 2024-09-09T05:25:25.513Z
Details:
{'is_contract': False,
 'to': '0xa3f27ae63b409f1e06be5665eba1f4002a71f54e',
 'value':

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
